In [1]:
import open3d as o3d
import copy
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
demo_icp_pcds = o3d.data.DemoICPPointClouds()
source = o3d.io.read_point_cloud(demo_icp_pcds.paths[0])
target = o3d.io.read_point_cloud(demo_icp_pcds.paths[1])

# function to visualize the point clouds after the transformation has been applied
def draw_registration_result(source, target, transformation):
    """
    param: source - source point cloud
    param: target - target point cloud
    param: transformation - 4 X 4 homogeneous transformation matrix
    """
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4459,
                                      front=[0.9288, -0.2951, -0.2242],
                                      lookat=[1.6784, 2.0612, 1.4451],
                                      up=[-0.3402, -0.9189, -0.1996])

In [3]:
o3d.visualization.draw_geometries([source, target],
                                      zoom=0.4459,
                                      front=[0.9288, -0.2951, -0.2242],
                                      lookat=[1.6784, 2.0612, 1.4451],
                                      up=[-0.3402, -0.9189, -0.1996])


[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [4]:
# ICP function which returns the transformation between the source and the desination point clouds
def ICP(source, target):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    target_te = copy.deepcopy(target)
    target_te = np.asarray(target_te.points)
    N = len(target_te)
    H = np.eye(4)
    prev_cost = 100000
    curr_cost = 0
    diff_cost = prev_cost - curr_cost
    while (diff_cost > 0.01):
        source_points = []
        source_tree = o3d.geometry.KDTreeFlann(source_temp)
        for i in range(N):
            [k, idx, _] = source_tree.search_knn_vector_3d(target_temp.points[i], 1)
            source_points.append(source_temp.points[idx[0]])

        source_points = np.asarray(source_points)
        target_points = np.asarray(target_temp.points)
        n = len(source_points)
    
        demean_s = np.transpose(source_points) - (np.sum(source_points, axis = 0) / n).reshape(3, 1)
        demean_d = np.transpose(target_points) - (np.sum(target_points, axis = 0) / n).reshape(3, 1)

        U, _, Vt = np.linalg.svd(demean_d @ np.transpose(demean_s))
        
        R = U @ Vt
        source_points = np.transpose(source_points)
        target_points = np.transpose(target_points)
        T = (np.sum(target_points, axis = 1) - (R @ np.sum(source_points, axis = 1))) / n

        curr_cost = np.linalg.norm(demean_d - (R @ demean_s))
        diff_cost = abs(prev_cost - curr_cost)
        prev_cost = curr_cost
        print(curr_cost)
        H_now = np.empty((4, 4))
        H_now[:3, :3] = R
        H_now[:3, 3] = T
        H_now[3, :] = [0, 0, 0, 1]

        H = H_now @ H
        source_temp.transform(H_now)
    return H

In [5]:
transformation = ICP(source, target)

63.0501322102132
58.590566379614884
56.65251136083426
55.51010875335899
54.65908094114768
53.89855837115818
53.19447689400112
52.50915748178347
51.82606798883577
51.12062634491721
50.370085260204945
49.50942878500292
48.47860750353497
47.273821143265906
45.88401262287705
44.3759440872945
42.90147406742706
41.49759170783179
40.148476479393224
38.882078158185195
37.74524930222672
36.73239426250806
35.83598419379687
34.954367319852096
34.01826316504208
33.07076038449125
32.229288487721256
31.45913599057241
30.609454099604335
29.58857230630831
28.31066146892175
26.936212186540725
25.650151274521722
24.264383899413033
22.541816928669622
20.526734745304093
18.443379859060386
16.55300218207878
14.773035491216714
13.222654820459889
11.927150997965493
10.850004306410222
10.005114701357405
9.369466554362399
8.86879855089762
8.461146662937926
8.153908233695544
7.930939044652796
7.7714168733035125
7.663043826787193
7.588046044977671
7.533185276023297
7.49615853165223
7.472302427659125
7.4569684487

In [7]:
draw_registration_result(source, target, transformation)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
